In [76]:
import pandas as pd
import json
from openai import OpenAI
from fpdf import FPDF 
import os
import requests
from datetime import datetime, timedelta
import pytz
import unicodedata
from secret import OPENAI_API_KEY # Remplacer par un fichier secret.py avec la clé api dedans
from zoneinfo import ZoneInfo
import fitz  # PyMuPDF
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText

In [77]:
def envoyer_pdf_par_mail(pdf_file_path, destinataire, sujet="Synthèse perturbations", message="Veuillez trouver en pièce jointe le rapport PDF."):

    expediteur = "salim.mansouri15@gmail.com"
    mot_de_passe = "fqum hxmq agup saks" 

    # Création du message
    msg = MIMEMultipart()
    msg['From'] = expediteur
    msg['To'] = destinataire
    msg['Subject'] = sujet

    # Corps du message
    msg.attach(MIMEText(message, 'plain'))

    # Ajout de la pièce jointe (PDF)
    with open(pdf_file_path, 'rb') as f:
        part = MIMEApplication(f.read(), Name=pdf_file_path)
        part['Content-Disposition'] = f'attachment; filename="{pdf_file_path}"'
        msg.attach(part)

    # Connexion SMTP sécurisée (ex: Gmail)
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(expediteur, mot_de_passe)
        server.send_message(msg)
        server.quit()
        print(f"Mail envoyé à {destinataire}")
    except Exception as e:
        print(f"Échec de l'envoi : {e}")

In [78]:
def extraire_contenu_pdf_modele(fichier_pdf_exemple):
    doc = fitz.open(fichier_pdf_exemple)
    contenu = ""
    for page in doc:
        contenu += page.get_text()
    return contenu.strip()

modele_pdf = extraire_contenu_pdf_modele("Note de situation.pdf")

In [79]:
df_ligne = pd.read_csv("referentiel-des-lignes.csv", delimiter=";")
df_ligne = df_ligne[df_ligne['TransportMode'] != 'bus']
df_ligne = df_ligne[['ID_Line', 'Name_Line', 'TransportMode']]
df_ligne.head()

,ID_Line,Name_Line,TransportMode
214,C01376,6,metro
215,C01380,10,metro
216,C01387,7B,metro
217,C01388,ORLYVAL,rail
218,C01389,T1,tram


In [80]:
# Fonction pour appeler l'API SIRI
def call_siri_api():
    """
    Appel à l'API SIRI pour récupérer les messages généraux
    """
    url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?LineRef=ALL"

    TOKEN = "HSv2mYkd7MbZ2rkiVTqE3Gh3AAiMh9SA"
    
    try:
        # Headers - ajoutez ici vos clés d'API si nécessaires
        headers = {
            "apiKey": TOKEN,  # Décommentez et ajoutez votre clé si nécessaire
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
        
        response = requests.get(url, headers=headers)
        
        # Vérifier si la requête a réussi
        if response.status_code == 200:
            return response.text
        else:
            print(f"Erreur lors de l'appel API: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Exception lors de l'appel API: {e}")
        return None

In [81]:
# Fonction pour extraire les données du message SIRI avec filtre sur les 15 dernières minutes
def extract_siri_data(json_data):
    """
    Extrait les informations pertinentes du message SIRI et les formate dans un dataframe
    Filtre seulement les messages créés dans les 15 dernières minutes
    """
    # Charger les données JSON
    data = json.loads(json_data)
    
    # Vérifier si les données ont le format attendu
    if not data.get('Siri') or not data['Siri'].get('ServiceDelivery') or not data['Siri']['ServiceDelivery'].get('GeneralMessageDelivery'):
        print("Le format des données SIRI n'est pas celui attendu.")
        return pd.DataFrame()
    
    # Liste pour stocker les données extraites
    rows = []
    
    # Calculer le timestamp d'il y a 15 minutes
    now = datetime.now(pytz.timezone('Europe/Paris'))
    fifteen_minutes_ago = now - timedelta(minutes=15)

    print("Heure actuel :", now)
    print('15 min :', fifteen_minutes_ago)
    
    # Parcourir les messages
    for message_delivery in data['Siri']['ServiceDelivery']['GeneralMessageDelivery']:
        if 'InfoMessage' in message_delivery:
            for info in message_delivery['InfoMessage']:
                # Extraire le timestamp d'enregistrement
                recorded_time_str = info.get('RecordedAtTime', '')
                
                # Vérifier si le message est récent (moins de 15 minutes)
                if recorded_time_str:
                    try:
                        # Convertir le timestamp ISO en objet datetime
                        recorded_time = datetime.fromisoformat(recorded_time_str.replace('Z', '+00:00'))
                        
                        # Skip si le message est plus ancien que 15 minutes
                        if recorded_time < fifteen_minutes_ago:
                            continue
                            
                    except (ValueError, TypeError) as e:
                        print(f"Erreur lors du parsing de la date: {e}, date: {recorded_time}")
                        continue
                
                # Extraire les informations nécessaires
                valid_until = info.get('ValidUntilTime', '')

                
                line_ref = "Non spécifiée"
                # Extraire la ligne concernée (si disponible)
                if 'Content' in info and 'LineRef' in info['Content'] and info['Content']['LineRef']:
                    line_ref = info['Content']['LineRef'][0].get('value', '')

                    if 'STIF:Line::' in line_ref:
                        line_ref = line_ref.split('STIF:Line::')[1].strip(':')
                    

                # Catégorie (type de message)
                categorie = info.get('InfoChannelRef', {}).get('value')
                
                # Texte du message
                texte = ""
                if 'Content' in info and 'Message' in info['Content'] and info['Content']['Message']:
                    message_text = info['Content']['Message'][0].get('MessageText', {})
                    if message_text:
                        texte = message_text.get('value', '')
                
                # Convertir les dates ISO en format plus lisible
                try:
                    heure_locale = recorded_time.astimezone(ZoneInfo("Europe/Paris"))
                    date_debut = heure_locale.strftime('%Y-%m-%d %H:%M')
                except (ValueError, TypeError):
                    date_debut = recorded_time_str
                
                try:
                    date_fin = datetime.fromisoformat(valid_until.replace('Z', '+00:00')).astimezone(ZoneInfo("Europe/Paris")).strftime('%Y-%m-%d %H:%M')
                except (ValueError, TypeError):
                    date_fin = valid_until
                
                # Ajouter à notre liste de données
                rows.append({
                    "Lignes": line_ref,
                    "Date de début": date_debut,
                    "Date de fin": date_fin,
                    "Catégorie": categorie,
                    "Texte de l'ICV gare et bord": texte
                })
    
    # Créer un dataframe avec nos données
    df = pd.DataFrame(rows)
    if df.empty:
        print("Aucun message créé dans les 15 dernières minutes n'a été trouvé.")
    else:
        print(f"Nombre de messages récents trouvés: {len(df)}")
    
    return df

In [82]:
# Code complet d'intégration avec le code précédent
def process_siri_to_pdf(output_pdf="Synthese_Perturbations_SIRI.pdf"):
    """
    Appelle l'API SIRI, traite les données et génère un PDF de synthèse
    des messages des 15 dernières minutes
    """
    # 1. Appeler l'API SIRI
    print("Appel de l'API SIRI...")
    siri_json_data = call_siri_api()
    
    if not siri_json_data:
        print("Échec de l'appel API.")
        return
    
    print("Données reçues de l'API SIRI.")
    
    # 2. Extraire les données du JSON SIRI (uniquement les 15 dernières minutes)
    df = extract_siri_data(siri_json_data)

    # Vérifier si le dataframe est vide
    if df.empty:
        print("Aucun message récent trouvé. Traitement arrêté.")
        return "Pas de nouveau PDF généré - aucune perturbation récente."

    print("\ndf API :")
    print(df)

    print("\ndf ref :")
    print(df_ligne.head())

    # Merge des deux dataframes sur les IDs
    final_df = df.merge(df_ligne, how='left', left_on='Lignes', right_on='ID_Line')

    # Remplacer NaN par "Non spécifiée" dans Name_Line uniquement si Lignes vaut "Non spécifiée"
    final_df['Name_Line'] = final_df.apply(lambda row: "Non spécifiée" if row['Lignes'] == "Non spécifiée" else row['Name_Line'],axis=1)

    final_df['TransportMode'] = final_df.apply(lambda row: "non_specifie" if row['Lignes'] == "Non spécifiée" else row['TransportMode'],axis=1)

    # Supprimer les lignes dont l’ID de ligne n'est pas reconnu
    final_df = final_df[(final_df['Name_Line'].notna()) | (final_df['Name_Line'] == "Non spécifiée")].copy()

    # Supprimer l’ID en double
    final_df.drop(columns=['ID_Line'], inplace=True)

    print("\ndf finale :")
    print(final_df.head(), "\n")
    
    if final_df.empty:
        print("Aucune donnée pertinente n'a été extraite du message SIRI (15 dernières minutes).")
        return
        
    print(f"Données extraites: {len(final_df)} messages des 15 dernières minutes")
    
    # 3. Générer un prompt pour OpenAI
    def generate_prompt(dataframe):
        prompt = (
            "Tu es un assistant de la SNCF. À partir des données suivantes extraites de messages SIRI "
            "des 15 dernières minutes, rédige une synthèse claire, concise et structurée à destination du directeur opérationnel.\n\n"
        )

        prompt += "Utilise comme **référence de ton style** la synthèse suivante extraite d’un PDF précédent :\n\n"
        prompt += modele_pdf + "\n\n"

        prompt += (
            "Structure ta synthèse de la façon suivante :\n"
            "- Réseau **Train-RER**\n"
            "- Réseau **Métro**\n"
            "- Réseau **Tram-Train**\n"

            "Si une **ligne** ou un **mode de transport** est marqué comme *Non spécifiée*, "
            "essaye de le **déduire à partir du texte du message** (gares citées, tronçons, noms de ligne, RER, etc.). "
            "Classe l'information dans le bon réseau si la déduction est fiable.\n\n"

            "Voici les données brutes à utiliser :\n\n"
        )

        # Regrouper par type de réseau
        mapping = {
            "rail": "Train-RER",
            "metro": "Métro",
            "tram": "Tram-Train",
            "non_specifie": "Non spécifié"
        }

        # S'assurer que toutes les colonnes nécessaires existent
        if 'TransportMode' not in dataframe.columns:
            # Si TransportMode n'existe pas, ajouter une colonne par défaut
            dataframe['TransportMode'] = 'non_specifie'

        for mode_technique in ['rail', 'metro', 'tram', 'non_specifie']:
            bloc_df = dataframe[dataframe['TransportMode'] == mode_technique]
            if bloc_df.empty:
                continue
            prompt += f"--- Réseau {mapping[mode_technique]} ---\n"
            for _, row in bloc_df.iterrows():
                texte = row["Texte de l'ICV gare et bord"].strip().replace('\n', ' ')
                prompt += (
                    f"Ligne {row['Name_Line']} | {row['Catégorie']} | "
                    f"{row['Date de début']} - {row['Date de fin']} | {texte}\n"
                )
            prompt += "\n"

        prompt += (
            "\nRédige maintenant la synthèse structurée comme indiqué, suivie de l'analyse des impacts inter-lignes.\n"
            "N'invente rien. Tu peux regrouper les perturbations similaires.\n"
        )

        return prompt

    prompt_text = generate_prompt(final_df)
    print("Prompt généré pour OpenAI")
    
    # 4. Appel à l'API OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Tu es un assistant expert en rédaction opérationnelle pour la SNCF."},
            {"role": "user", "content": prompt_text}
        ],
        temperature=0.4,
        max_tokens=1200
    )

    synthese_text = response.choices[0].message.content
    print("Synthèse générée par OpenAI")
    
    # 5. Génération du PDF
    def enregistrer_pdf_alternative(nom_fichier, contenu_texte):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', '', 12)

        # Remplacer les caractères problématiques
        replacements = {
            '\u2022': '-', '\u2019': "'", '\u2018': "'",
            '\u201c': '"', '\u201d': '"', '\u2013': '-', '\u2014': '--', '\u2026': '...',
            '\u2006': ' ',  # espace fine
        }

        for old, new in replacements.items():
            contenu_texte = contenu_texte.replace(old, new)

        # Supprimer ou normaliser les caractères non Latin-1 restants
        def to_latin1(text):
            return unicodedata.normalize('NFKD', text).encode('latin-1', 'ignore').decode('latin-1')

        contenu_texte = to_latin1(contenu_texte)

        for ligne_contenu in contenu_texte.split('\n'):
            pdf.multi_cell(0, 10, ligne_contenu)

        pdf.output(nom_fichier)

    # Générer le PDF
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        output_pdf_name = f"Synthese_Perturbations_{timestamp}.pdf"
        enregistrer_pdf_alternative(output_pdf_name, synthese_text)
        print(f"PDF généré avec succès: {output_pdf_name}")
    except Exception as e:
        print(f"Erreur lors de la génération du PDF: {e}")
        
    return output_pdf_name

In [83]:
# def tts(file):
#     now = datetime.now(pytz.timezone('Europe/Paris'))
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

#     with open(file, "r") as f:
#         txt_content = f.read()

#     tts.tts_to_file(text = txt_content, speaker_wav="my/cloning/les-lectures-de-simone-un-nouveau-pneu.wav", 
#                     language="fr", file_path=f"output.wav_{now}",speed=0.9)


In [84]:
# Pour exécuter le processus complet
if __name__ == "__main__":
    try:
        pdf_file = process_siri_to_pdf()
        print(f"Traitement terminé. Fichier PDF: {pdf_file}")
        # Sujet avec date et heure locale
        heure_point = datetime.now(pytz.timezone("Europe/Paris"))
        sujet_automatique = heure_point.strftime("Point de situation - %d/%m/%Y à %Hh%M")
        # Envoi par mail
        envoyer_pdf_par_mail(
            pdf_file_path=pdf_file,
            destinataire="salim.mansouri15@gmail.com",
            sujet=sujet_automatique,
            message="Bonjour,\n\nVoici la synthèse des perturbations SNCF des 15 dernières minutes.\n\nCordialement."
        )
        #tts(pdf_file)
    except Exception as e:
        print(f"Erreur lors du traitement: {e}")

Appel de l'API SIRI...
Données reçues de l'API SIRI.
Heure actuel : 2025-04-10 01:46:21.617355+02:00
15 min : 2025-04-10 01:31:21.617355+02:00
Aucun message créé dans les 15 dernières minutes n'a été trouvé.
Aucun message récent trouvé. Traitement arrêté.
Traitement terminé. Fichier PDF: Pas de nouveau PDF généré - aucune perturbation récente.
Erreur lors du traitement: [Errno 2] No such file or directory: 'Pas de nouveau PDF généré - aucune perturbation récente.'
